<a href="https://colab.research.google.com/github/Jxiang2/CMPUT340/blob/main/Copy_of_Least_Squares_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Lab 2: Least Squares</h2>

<b>Demo Date: </b> Mar. 09 <br>

In this lab you will implement two algorithms for solving an ill-conditioned least squares problem. We create an artificial and overdetermined least-squares problem by removing two columns of a $10 \times 10$ Hilbert matrix, a classic ill-conditioned matrix. As we remove the two columns from the matrix it is no longer a Hilbert matrix, but it creates an overdetermined system with a large condition number: $\approx 3,796,554,168$.

Implement an algorithm that you believe will compute the value of $x$ for the least squares problem $Ax \approx b$ as accurately as the problem allows. You will also implement an algorithm that you believe to produce inaccurate solutions due to the large condition number of the system. In this lab you do not need to worry about having a fast implementation, any non-vectorized implementation will be enough. 

Compare the two solutions in terms of the norm of their residual vector $Ax - b$. In the demo of your lab you should be able to explain your choices of algorithms and the results you obtained. Why did one method produce a solution with a smaller residual than the other? 

In [ ]:
import numpy as np
import copy
import random
from scipy.linalg import hilbert
from scipy.optimize import lsq_linear

b = np.ones(10).reshape(10,1)
A = hilbert(10)
A = np.delete(A, 5, 1)
A = np.delete(A, 4, 1)
#A is 10X8
print('Condition number of A: ', np.linalg.norm(A) * np.linalg.norm(np.linalg.pinv(A)))

# Use this test instance while developing the algorithms. 
# This instance is much easier to debug than the ill-conditioned problem above. 
A_test = np.array([[1, 2, 2], [4, 4, 2], [4, 6, 4]], dtype=float)
b_test = np.array([3, 6, 10], dtype=float)
b_test = b_test.reshape(3,1)


Condition number of A:  3796554182.3041143


In [ ]:
def forward_substituion(L, b):
  x = np.zeros(L.shape[0])
  n = len(x)
  
  for j in range(0, n ):
    if L[j][j] == 0:
      return None
    x[j] = b[j] / L[j][j]

    for i in range(j+1, n):
      b[i] = b[i] - L[i][j] * x[j]
    
  return x

def back_substituion(U, b):
  x = np.zeros(U.shape[0])
  n = len(x)
  
  for j in range(n - 1, -1, -1):
    if U[j][j] == 0:
      return None
    x[j] = b[j] / U[j][j] 

    for i in range(0, j):
      b[i] = b[i] - U[i][j] * x[j]

  return x
      

def lu_factor_v1(A):
  M = np.identity(A.shape[0])
  n = A.shape[0]
  
  for k in range(0, n-1):
    if A[k][k] == 0:
      return None

    for i in range(k+1, n):
      M[i][k] = A[i][k]/A[k][k]
  
    for j in range(k, n):
      for i in range(k+1, n):
        A[i][j] = A[i][j] - M[i][k] * A[k][j]

    

  return M, A

In [ ]:
def inaccurate_method(A,b):
  #use the fact that residual is perpendicular to Ax
  #A.T(b-AX) = 0 ==> ATAx = ATb
  A1 = copy.deepcopy(A.T.dot(A))
  b1 = copy.deepcopy(A.T.dot(b))
  L, U = lu_factor_v1(A1)

  y = forward_substituion(L, b1)
  x = back_substituion(U, y)
  return x

x1= inaccurate_method(A, b)
print(x1)

[ 4.55237856e+00 -1.50338192e+02  1.03217498e+03 -1.99256355e+03
  5.81193932e+03  1.29245482e+03 -1.55360867e+04  9.60097244e+03]


In [ ]:
def Householder_QR_factor(A,b):
  #general idea: preserve norm of col vectors to a orthogonal basis

  #append b to A as the last col
  Ab = np.concatenate((A,b),axis=1)

  #A test of QR factorization, should retrun [[-2, -3, -2], [0, -5, -2], [0, 0, -4], [0, 0, 0]]
  #Ab = np.array([[1, -1, 4], [1, 4, -2], [1, 4, 2], [1, -1, 0]], dtype=float)

  m = Ab.shape[0]   #row
  n = Ab.shape[1]   #col
  #print("row: ",m,"col: ",n)

  for k in range(min(n,m-1)): 

    v = np.zeros_like(Ab[:,k])
    #print(v)
    for i in range(len(v)):
      if i < k:
        #transfer weights of entries below Abkk, so later transformation the norms wont touch the entries above akk
        v[i] = 0
      else:
        v[i] = Ab[:,k][i]
    
    alpha = np.linalg.norm(v, ord=2)
    #avoid cancellation error
    if alpha > 1:
      alpha = -alpha
    #print(alpha)

    e = np.zeros_like(Ab[:,k])
    e[k] = 1
    #compute householder vector v, v = a - |a|e1
    v = v.T - alpha*e
    #print(v)
    beta = v.T.dot(v)

    if beta == 0:
      #akk = 0, all values we want to make to 0 are already zeros, as well as akk. ==>rank deficient
      continue
    
    for j in range(k,n):
      gamma = v.T.dot(Ab[:,j])
      #perform househoulder transformation to the curret col(remove the componets other than ek to ek, and preserve norm. Project 2 times), and later col to keep consistency.
      Ab[:,j] = Ab[:,j] - 2*(gamma/beta)*v

  return Ab


Ab_test = Householder_QR_factor(A,b)
print(Ab_test)
print("######################################################################################")
#seperate the R matrix and c1 vector
row_num = Ab_test.shape[0]
col_num = Ab_test.shape[1]
R0 = Ab_test[: , 0:col_num-1]
c = Ab_test[: , col_num-1:col_num]

#make R nxn and c1 nx1
A_col_num = A.shape[1]
r_row_num = R0.shape[0]

zeros = abs(r_row_num - A_col_num)
R = R0[:-zeros, :]
print(R)
c1 = c[:-zeros, :]
print(c1)
#c2 = c[A_col_num:, :]
#print(c2)
print("######################################################################################")

#solve Rx=c1
L, U = lu_factor_v1(R)
y = forward_substituion(L, c1)
x2 = back_substituion(U, y)
print(x2)

[[-1.24489667e+00 -7.30254107e-01 -5.32476953e-01 -4.23641004e-01
  -2.67524111e-01 -2.38949593e-01 -2.16053806e-01 -1.97267318e-01
  -2.35278021e+00]
 [ 0.00000000e+00  1.57356709e-01  1.76816009e-01  1.72683318e-01
   1.42790794e-01  1.33686109e-01  1.25472117e-01  1.18094576e-01
   1.91761724e+00]
 [ 0.00000000e+00  1.38777878e-17  1.34548169e-02  2.21489714e-02
   3.13864772e-02  3.19635984e-02  3.20187091e-02  3.17474364e-02
   8.43119294e-01]
 [ 0.00000000e+00  1.38777878e-17  0.00000000e+00  9.38887391e-04
   3.75636504e-03  4.35983575e-03  4.81743670e-03  5.15740910e-03
   2.67891709e-01]
 [ 0.00000000e+00  6.93889390e-18  0.00000000e+00  1.69406589e-21
   2.49745716e-04  3.54165627e-04  4.51533946e-04  5.38737018e-04
   6.65787384e-02]
 [ 0.00000000e+00  6.93889390e-18  0.00000000e+00  2.71050543e-20
   2.71050543e-20  4.54712004e-06  1.07372606e-05  1.78960757e-05
   1.08815446e-02]
 [ 0.00000000e+00  6.93889390e-18  0.00000000e+00  5.42101086e-20
   2.71050543e-20  0.0000000

In [ ]:
#|dx/x| <= cond(A)*1/cos(theta)*|db/b|

#normal equation method would be problematic if Ax is normal to b because small pertubation of b cause large pertubation of the solution Ax.
#A.T.dot(A) can cause prblems because hilbert matrix contain small entires and A.T.dot(A) makes the entry smaller, make A.T.dot(A) a singular ==> inflates Cond(A), as Cond(ATA) = Cond(A)^2

#QR factor no need to do A.T.dot(A)

res1 = A.dot(x1) - b
res2 = A.dot(x2) - b

res1_norm = np.linalg.norm(res1, ord=2)
res2_norm = np.linalg.norm(res2, ord=2)
print("res1 norm: ", res1_norm)
print("res2 norm: ", res2_norm)

#The solution obtained by QR factorization has less residual

res1 norm:  0.0004349704658567487
res2 norm:  4.1409549769969474e-05
